---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [46]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    # open text file
    f = open('university_towns.txt','r')
    # save into list, remove \n
    utowns = [line.strip() for line in f]
    utowns
    # base state and regions initialized to nothing
    s = ''
    r = ''
    # make a resultant dataframe
    data = pd.DataFrame(columns = ['State','RegionName'])
    # loop through list
    for i in range(0, len(utowns)):
        # lines with [edit] are states
        if '[edit]' in utowns[i]:
            # take out everything after '[...'
            sep = '['
            s = utowns[i].split(sep, 1)[0].strip()
        # lines without are regions
        else:
            # take out everything after '(...'
            sep = '('
            r = utowns[i].split(sep, 1)[0].strip()
            # add the state/region entry
            data = data.append({'State': s, 'RegionName' : r}, ignore_index = True)
    return data

In [30]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    # read in GDP, start at 2000
    GDP = pd.read_excel('gdplev.xls', skiprows = 219)
    # drop by-year columns, only want quarterly
    GDP.drop(GDP.columns[[0,1,2,3,7]], axis=1, inplace=True)
    # rename three columns we need
    GDP.columns = ['Quarter', 'GDP', 'GDP2009']
    for i in range(0, len(GDP) - 2):
        # recession starts when GDP declines 2 years in a row
        if GDP.iloc[i]['GDP2009'] > GDP.iloc[i + 1]['GDP2009'] and GDP.iloc[i + 1]['GDP2009'] > GDP.iloc[i + 2]['GDP2009'] :
            # print(GDP.iloc[i], '\n', GDP.iloc[i + 1], '\n', GDP.iloc[i + 2], '\n')
            # stop loop, found start
            break
    # year at the start of decline (one after local peak)
    return GDP.iloc[i + 1]['Quarter']

In [4]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    # read in GDP, start at 2000
    GDP = pd.read_excel('gdplev.xls', skiprows = 219)
    # drop by-year columns, only want quarterly
    GDP.drop(GDP.columns[[0,1,2,3,7]], axis=1, inplace=True)
    # rename three columns we need
    GDP.columns = ['Quarter', 'GDP', 'GDP2009']
    # get the quarter that recession started in
    rec_start = get_recession_start()
    # get the index that the recession started in
    rec_start_index = GDP[GDP['Quarter'] == rec_start].index[0]
    # limit to recession start onward
    GDP = GDP[rec_start_index:]
    GDP
    for i in range(0, len(GDP) - 2):
        # recession ends when GDP raises 2 years in a row
        if GDP.iloc[i]['GDP2009'] < GDP.iloc[i + 1]['GDP2009'] and GDP.iloc[i + 1]['GDP2009'] < GDP.iloc[i + 2]['GDP2009'] :
            # print(GDP.iloc[i], '\n', GDP.iloc[i + 1], '\n', GDP.iloc[i + 2], '\n')
            # stop loop, found start
            break
    # end of recession
    return GDP.iloc[i + 2]['Quarter']

In [5]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    # read in GDP, start at 2000
    GDP = pd.read_excel('gdplev.xls', skiprows = 219)
    # drop by-year columns, only want quarterly
    GDP.drop(GDP.columns[[0,1,2,3,7]], axis=1, inplace=True)
    # rename three columns we need
    GDP.columns = ['Quarter', 'GDP', 'GDP2009']
    # get the quarter that recession started in
    rec_start = get_recession_start()
    # get the index that the recession started in
    rec_start_index = GDP[GDP['Quarter'] == rec_start].index[0]
    # get the quarter that recession ended in
    rec_end = get_recession_end()
    # get the index that the recession ended in
    rec_end_index = GDP[GDP['Quarter'] == rec_end].index[0]
    # limit to recession duration
    GDP = GDP[rec_start_index:rec_end_index]
    # find the index of the minimum GDP
    GDPmin = GDP['GDP2009'].idxmin()
    # find the quarter of that index
    return GDP.loc[GDPmin]['Quarter']

In [6]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    city = pd.read_csv('City_Zhvi_AllHomes.csv')
    # city.drop(city.loc[:,'1996-04':'1999-12'], inplace = True, axis = 1)
    cityinfo = city[['State', 'RegionName']]
    cityinfo = cityinfo.replace({"State": states})
    # year 2000 and on
    cityquarters = city.loc[:, '2000-01':]
    cityquarters
    # convert to quarters
    cityquarters = (cityquarters.groupby(pd.PeriodIndex(cityquarters.columns, freq='Q'), axis=1)
                                .mean()
                                .rename(columns=lambda c: str(c).lower()))
    city = pd.concat([cityinfo, cityquarters], axis = 1)
    city = city.set_index(['State', 'RegionName'])
    return city


In [1]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    # read in GDP, start at 2000
    GDP = pd.read_excel('gdplev.xls', skiprows = 219)
    # drop by-year columns, only want quarterly
    GDP.drop(GDP.columns[[0,1,2,3,7]], axis=1, inplace=True)
    # rename three columns we need
    GDP.columns = ['Quarter', 'GDP', 'GDP2009']
    # get the quarter that recession started in
    rec_start = get_recession_start()
    # get the index one quarter before recession start
    before_rec_index = GDP[GDP['Quarter'] == rec_start].index[0] - 1
    before_rec_q = GDP['Quarter'].iloc[before_rec_index]
    rec_bottom = get_recession_bottom()

    # get university towns and cities and find the desired ratio
    utown = get_list_of_university_towns().set_index(['State','RegionName'])
    city = convert_housing_data_to_quarters()
    city['Ratio'] = city[before_rec_q] / city[rec_bottom]
    
    # make dfs for only university towns and non-university towns
    onlyutown = pd.merge(utown, city, how='inner', left_index = True, right_index = True)
    nonutown = city[~city.index.isin(utown.index)]
    
    # t-test, remove nas
    res = ttest_ind(onlyutown['Ratio'], nonutown['Ratio'], nan_policy = 'omit')
    
    # check if results are significant (p-val < .01)
    different = False
    if(res[1] < .01):
        different = True
    # check which had a lower ratio
    better = ''
    if(onlyutown['Ratio'].mean() < nonutown['Ratio'].mean()):
        better = 'university town'
    else:
        better = 'non-university town'
    return (different, res[1], better)